In [0]:
# from google.colab import drive 
# drive.mount('/content/gdrive')

In [0]:
# root_path = '/content/gdrive/My Drive/Colab Notebooks/godl/'
# %cd /content/gdrive/My\ Drive/Colab\ Notebooks/godl/
# !cp train\ \(9\).zip /content/.
# # !cp test_ApKoW4T.csv /content/.
# %cd /content
# !unzip train\ \(9\).zip;

In [0]:
import os

In [0]:
DATA_DIR = '/content/'
IMAGE_DIR = os.path.join(DATA_DIR, "images/")
SNAPSHOT_DIR = "/content/gdrive/My Drive/godl_submission/model_snapshot/"
OUTPUT_DIR = "/content/gdrive/My Drive/godl_submission/preds/"

In [0]:
from fastai import *
from fastai.vision import *

In [0]:
import sys
import os
import numpy as np
import pandas as pd
import shutil
import os
import math

In [0]:
from sklearn.metrics import f1_score, accuracy_score

In [0]:
df = pd.read_csv('train.csv', dtype='str')
df_test = pd.read_csv('test_ApKoW4T.csv', dtype='str')

In [0]:
tsms = ([
#     crop_pad(row_pct=(0.5,1), col_pct=(0.5,1), padding_mode='reflection', p=.75),
#     flip_lr(p=.75),
    symmetric_warp(magnitude=(-0.2, 0.2), p=.75),
    perspective_warp(magnitude=(-0.2, 0.2), p=.75),
    rotate(degrees = (-20, 20), p=.75),
#     zoom(scale=(1, 1.3), p=.75),
#     brightness(change=(.4, .6), p=.75),
#     contrast(scale=(.8,1.25), p=.75),
#     cutout(p=.5),
#     squish(scale=(.8,1.25), p=.75),
#     jitter(magnitude=(-0.01, 0.01), p=.5),
#     rgb_randomize(p=.75)
#     dihedral(p=.75),
],[])


In [0]:
def get_data_set(size=224, batch_size=32, tsms=tsms):
    np.random.seed(100)
    if tsms is None:
        tsms = ([],[])
    data = ImageDataBunch.from_df(path=IMAGE_DIR, df=df, valid_pct=0.2, ds_tfms=tsms, size=size, 
                                  resize_method=ResizeMethod.SQUISH, padding_mode='reflection',
                                  bs=batch_size, no_check=True).normalize(imagenet_stats)
    test = ImageList.from_df(df_test, path=IMAGE_DIR)
    data.add_test(test)
    return data

    

In [0]:
data = get_data_set(size=(128,128), batch_size=32, tsms=None)

In [0]:
data.classes, data.c, data.batch_size

In [0]:
head_reg4 = nn.Sequential(
    AdaptiveConcatPool2d(),
    Flatten(),
    nn.Linear(3840, 2048),
    nn.ReLU(),
    nn.BatchNorm1d(2048),
    nn.Dropout(0.5),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.5),
    nn.Linear(512, 5),
)
learn = cnn_learner(data, models.densenet201, metrics=[accuracy], custom_head=head_reg4)

In [0]:
# generating optimized TTA predictions and optimized randomized TTA predictions
for snapshot, scale in [
    ('dense201-224-9784.pth', 1.05),
    ('dense201-224-9768.pth', 1.05),
    ('dense201-224-9760.pth', 1.10),
    ('dense201-320-9744.pth', 1.10),
    ('dense201-256-9752.pth', 1.15),
    ('dense201-256-9760.pth', 0.90),
    ('dense201-256-9736.pth', 0.90),
    ('dense201-192-9704.pth', 1.15),
    ('dense201-224-9712.pth', 1.15),
    ('dense201-224-9688.pth', 1.20),
]:
    size = int(snapshot.split('-')[1])
    print (snapshot, size)
    
    learn.data = get_data_set(size=(size, size), batch_size=32, tsms=None)
    learn.load(SNAPSHOT_DIR + snapshot.strip('.pth'))
    
    test_probs_tta, _ = learn.TTA(scale=scale, ds_type=DatasetType.Test)  
    
    learn.data = get_data_set(size=(size, size), batch_size=32, tsms=tsms)
    test_probs_tta_random, _ = learn.TTA(scale=scale, ds_type=DatasetType.Test) 
    
    np.savetxt(OUTPUT_DIR + '{}-{}-{}.txt'.format(snapshot, 'test-tta', str(scale).replace('.','')), test_probs_tta.data.cpu().numpy())  
    np.savetxt(OUTPUT_DIR + '{}-{}-{}-random1.txt'.format(snapshot, 'test-tta', str(scale).replace('.','')), test_probs_tta_random.data.cpu().numpy())  
  

In [0]:
test_preds = []
for f in [
    'dense201-224-9784.pth-test-tta-105.txt',
    'dense201-224-9768.pth-test-tta-105.txt',
    'dense201-224-9760.pth-test-tta-11.txt',
    'dense201-320-9744.pth-test-tta-11.txt',
    'dense201-256-9752.pth-test-tta-115.txt',
    'dense201-256-9760.pth-test-tta-09.txt',
    'dense201-256-9736.pth-test-tta-09.txt',
    'dense201-192-9704.pth-test-tta-115.txt',
    'dense201-224-9712.pth-test-tta-115.txt',
    'dense201-224-9688.pth-test-tta-12.txt',
    'dense201-224-9784.pth-test-tta-105-random1.txt',
    'dense201-224-9768.pth-test-tta-105-random1.txt',
    'dense201-224-9760.pth-test-tta-11-random1.txt',
    'dense201-320-9744.pth-test-tta-11-random1.txt',
    'dense201-256-9752.pth-test-tta-115-random1.txt',
    'dense201-256-9760.pth-test-tta-09-random1.txt',
    'dense201-256-9736.pth-test-tta-09-random1.txt',
    'dense201-192-9704.pth-test-tta-115-random1.txt',
    'dense201-224-9712.pth-test-tta-115-random1.txt',
    'dense201-224-9688.pth-test-tta-12-random1.txt',
]:
    test_preds.append(torch.from_numpy(np.loadtxt(OUTPUT_DIR + f)).float().cuda())


In [0]:
df_test['category'] = torch.stack(test_preds, dim=0).sum(dim=0).argmax(1) + 1

In [0]:
df_test.to_csv(OUTPUT_DIR + 'final_sub_with_random_tta.csv', index=False)